In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import keras as kk
from keras.layers import Dense
import logging

In [3]:
import pickle
from tqdm import tqdm, tqdm_notebook, _tqdm_notebook, tqdm_pandas

In [4]:
log_fmt = "[%(asctime)s] %(levelname)s in %(module)s: %(message)s"
logging.basicConfig(format=log_fmt, level=logging.INFO)

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
data = pd.read_csv('../data2/data_1219.txt', sep='\t')
data.head()

,day,follow_topic,freq,gender,hour,inter_topic,label,q_ans_count,q_diff_qa_days_max,q_diff_qa_days_mean,...,uid_enc_count,qid_enc_count,gender_count,freq_count,uf_c1_count,uf_c2_count,uf_c3_count,uf_c4_count,uf_c5_count,wk
0,3865,-0.122170,4.0,3,22,0.021705,0.0,NaN,NaN,NaN,...,0.000000,0.000369,1.000000,0.960571,0.022114,0.371930,0.218308,0.062702,1.0,1
1,3862,-0.015598,4.0,3,15,-0.182562,0.0,NaN,NaN,NaN,...,0.000004,0.000036,1.000000,0.960571,0.024451,0.111552,0.444739,0.002961,1.0,5
2,3867,0.000000,3.0,3,4,0.070942,0.0,32.0,13.0,9.53125,...,0.000022,0.001441,1.000000,0.290809,0.006221,0.034154,0.402727,0.044313,1.0,3
3,3861,-0.302475,5.0,2,20,-0.191518,0.0,3.0,234.0,222.33333,...,0.000011,0.000018,0.000000,1.000000,0.024451,0.111552,0.444739,0.002961,1.0,4
4,3862,-0.095008,4.0,1,8,-0.176807,1.0,NaN,NaN,NaN,...,0.000043,0.000027,0.008695,0.960571,0.005739,0.007515,0.438648,0.125989,1.0,5


In [7]:
feature_cols = [x for x in data.columns if x not in ('label', 'uid', 'qid', 'dt', 'day')]

In [8]:
train_label=2593669

In [9]:
data = data.fillna(0)
data.head()

,day,follow_topic,freq,gender,hour,inter_topic,label,q_ans_count,q_diff_qa_days_max,q_diff_qa_days_mean,...,uid_enc_count,qid_enc_count,gender_count,freq_count,uf_c1_count,uf_c2_count,uf_c3_count,uf_c4_count,uf_c5_count,wk
0,3865,-0.122170,4.0,3,22,0.021705,0.0,0.0,0.0,0.00000,...,0.000000,0.000369,1.000000,0.960571,0.022114,0.371930,0.218308,0.062702,1.0,1
1,3862,-0.015598,4.0,3,15,-0.182562,0.0,0.0,0.0,0.00000,...,0.000004,0.000036,1.000000,0.960571,0.024451,0.111552,0.444739,0.002961,1.0,5
2,3867,0.000000,3.0,3,4,0.070942,0.0,32.0,13.0,9.53125,...,0.000022,0.001441,1.000000,0.290809,0.006221,0.034154,0.402727,0.044313,1.0,3
3,3861,-0.302475,5.0,2,20,-0.191518,0.0,3.0,234.0,222.33333,...,0.000011,0.000018,0.000000,1.000000,0.024451,0.111552,0.444739,0.002961,1.0,4
4,3862,-0.095008,4.0,1,8,-0.176807,1.0,0.0,0.0,0.00000,...,0.000043,0.000027,0.008695,0.960571,0.005739,0.007515,0.438648,0.125989,1.0,5


In [12]:
print(feature_cols)

['follow_topic', 'freq', 'gender', 'hour', 'inter_topic', 'q_ans_count', 'q_diff_qa_days_max', 'q_diff_qa_days_mean', 'q_diff_qa_days_sum', 'q_has_img_max', 'q_has_img_mean', 'q_has_img_sum', 'q_has_video_max', 'q_has_video_mean', 'q_has_video_sum', 'q_inv_count', 'q_inv_mean', 'q_inv_std', 'q_inv_sum', 'q_is_dest_max', 'q_is_dest_mean', 'q_is_dest_sum', 'q_is_good_max', 'q_is_good_mean', 'q_is_good_sum', 'q_is_rec_max', 'q_is_rec_mean', 'q_is_rec_sum', 'q_reci_cheer_max', 'q_reci_cheer_mean', 'q_reci_cheer_sum', 'q_reci_comment_max', 'q_reci_comment_mean', 'q_reci_comment_sum', 'q_reci_dis_max', 'q_reci_dis_mean', 'q_reci_dis_sum', 'q_reci_mark_max', 'q_reci_mark_mean', 'q_reci_mark_sum', 'q_reci_no_help_max', 'q_reci_no_help_mean', 'q_reci_no_help_sum', 'q_reci_tks_max', 'q_reci_tks_mean', 'q_reci_tks_sum', 'q_reci_uncheer_max', 'q_reci_uncheer_mean', 'q_reci_uncheer_sum', 'q_reci_xxx_max', 'q_reci_xxx_mean', 'q_reci_xxx_sum', 'q_word_count_max', 'q_word_count_mean', 'q_word_count_su

In [13]:
# target编码
logging.info("feature size %s", len(feature_cols))
num_x = len(feature_cols)
X_train_all = data.iloc[:train_label][feature_cols]
y_train_all = data.iloc[:train_label]['label']
# test = data.iloc[len(train_label):]
# # del data
# assert len(test) == sub_size

# logging.info("train shape %s", train_label.shape)

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for index, (train_idx, val_idx) in enumerate(fold.split(X=X_train_all, y=y_train_all)):
    break

X_train, X_val, y_train, y_val = X_train_all.iloc[train_idx][feature_cols], X_train_all.iloc[val_idx][feature_cols], \
                                 y_train_all.iloc[train_idx], \
                                 y_train_all.iloc[val_idx]
del X_train_all



[2019-12-20 16:33:57,052] INFO in <ipython-input-13-4f6a3e4f2eb1>: feature size 128


In [15]:
X_train.head()

,follow_topic,freq,gender,hour,inter_topic,q_ans_count,q_diff_qa_days_max,q_diff_qa_days_mean,q_diff_qa_days_sum,q_has_img_max,...,uid_enc_count,qid_enc_count,gender_count,freq_count,uf_c1_count,uf_c2_count,uf_c3_count,uf_c4_count,uf_c5_count,wk
0,-0.122170,4.0,3,22,0.021705,0.0,0.0,0.00000,0.0,0.0,...,0.000000,0.000369,1.000000,0.960571,0.022114,0.371930,0.218308,0.062702,1.0,1
1,-0.015598,4.0,3,15,-0.182562,0.0,0.0,0.00000,0.0,0.0,...,0.000004,0.000036,1.000000,0.960571,0.024451,0.111552,0.444739,0.002961,1.0,5
2,0.000000,3.0,3,4,0.070942,32.0,13.0,9.53125,305.0,1.0,...,0.000022,0.001441,1.000000,0.290809,0.006221,0.034154,0.402727,0.044313,1.0,3
3,-0.302475,5.0,2,20,-0.191518,3.0,234.0,222.33333,667.0,1.0,...,0.000011,0.000018,0.000000,1.000000,0.024451,0.111552,0.444739,0.002961,1.0,4
4,-0.095008,4.0,1,8,-0.176807,0.0,0.0,0.00000,0.0,0.0,...,0.000043,0.000027,0.008695,0.960571,0.005739,0.007515,0.438648,0.125989,1.0,5


In [20]:
X_train_array = np.array(X_train)
y_train_array = np.array(y_train)
X_val_array = np.array(X_val)
y_val_array = np.array(y_val)

In [24]:
model = kk.models.Sequential()
model.add(Dense(200,input_dim=num_x,activation='relu'))
model.add(Dense(220,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(X_train_array,y_train_array,epochs=200,batch_size=240)

scores = model.evaluate(x=X_val_array,y=y_val_array)
print('\n%s :%.2f' %(model.metrics_names[1],scores[1]))

Epoch 1/200
2074934/2074934 [==============================] - 27s - loss: 2.5804 - acc: 0.8399    
Epoch 2/200
2074934/2074934 [==============================] - 27s - loss: 2.5791 - acc: 0.8400    
Epoch 3/200
2074934/2074934 [==============================] - 28s - loss: 2.5791 - acc: 0.8400    
Epoch 4/200
2074934/2074934 [==============================] - 28s - loss: 2.5791 - acc: 0.8400    
Epoch 5/200
2074934/2074934 [==============================] - 28s - loss: 2.5791 - acc: 0.8400    
Epoch 6/200
2074934/2074934 [==============================] - 27s - loss: 2.5791 - acc: 0.8400    
Epoch 7/200
2074934/2074934 [==============================] - 26s - loss: 2.5791 - acc: 0.8400    
Epoch 8/200
2074934/2074934 [==============================] - 26s - loss: 2.5791 - acc: 0.8400    
Epoch 9/200
2074934/2074934 [==============================] - 26s - loss: 2.5791 - acc: 0.8400    
Epoch 10/200
2074934/2074934 [==============================] - 26s - loss: 2.5791 - acc: 0.8400    

2074934/2074934 [==============================] - 27s - loss: 2.5791 - acc: 0.8400    
Epoch 83/200
2074934/2074934 [==============================] - 28s - loss: 2.5791 - acc: 0.8400    
Epoch 84/200
2074934/2074934 [==============================] - 28s - loss: 2.5791 - acc: 0.8400    
Epoch 85/200
2074934/2074934 [==============================] - 27s - loss: 2.5791 - acc: 0.8400    
Epoch 86/200
2074934/2074934 [==============================] - 27s - loss: 2.5791 - acc: 0.8400    
Epoch 87/200
2074934/2074934 [==============================] - 28s - loss: 2.5791 - acc: 0.8400    
Epoch 88/200
2074934/2074934 [==============================] - 28s - loss: 2.5791 - acc: 0.8400    
Epoch 89/200
2074934/2074934 [==============================] - 28s - loss: 2.5791 - acc: 0.8400    
Epoch 90/200
2074934/2074934 [==============================] - 28s - loss: 2.5791 - acc: 0.8400    
Epoch 91/200
2074934/2074934 [==============================] - 27s - loss: 2.5791 - acc: 0.8400    
Epo

2074934/2074934 [==============================] - 27s - loss: 2.5791 - acc: 0.8400    
Epoch 163/200
2074934/2074934 [==============================] - 27s - loss: 2.5791 - acc: 0.8400    
Epoch 164/200
2074934/2074934 [==============================] - 27s - loss: 2.5791 - acc: 0.8400    
Epoch 165/200
2074934/2074934 [==============================] - 27s - loss: 2.5791 - acc: 0.8400    
Epoch 166/200
2074934/2074934 [==============================] - 28s - loss: 2.5791 - acc: 0.8400    
Epoch 167/200
2074934/2074934 [==============================] - 28s - loss: 2.5791 - acc: 0.8400    
Epoch 168/200
2074934/2074934 [==============================] - 27s - loss: 2.5791 - acc: 0.8400    
Epoch 169/200
2074934/2074934 [==============================] - 27s - loss: 2.5791 - acc: 0.8400    
Epoch 170/200
2074934/2074934 [==============================] - 28s - loss: 2.5791 - acc: 0.8400    
Epoch 171/200
2074934/2074934 [==============================] - 28s - loss: 2.5791 - acc: 0.840